In [1]:
import numpy as np
import scipy as sp
from scipy.sparse.csgraph import connected_components

import networkx as nx
import igraph as g
import graph_tool as gt
from graph_tool.topology import label_components

In [2]:
tmp = nx.erdos_renyi_graph(66, 0.015)
a = nx.to_numpy_matrix(tmp)
A = sp.sparse.csr_matrix(a)

# 1) Using scipy.connected_components with dense matrix

In [3]:
%%timeit

for _ in range(100):
    _, labels = connected_components(a, directed=False)
    _, counts = np.unique(labels, return_counts=True)
    counts1 = -np.sort(-counts)

89.8 ms ± 8.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# 2) Using scipy.connected_components converting dense to sparse each time

In [4]:
%%timeit

for _ in range(100):
    A = sp.sparse.csr_matrix(a)
    _, labels = connected_components(A, directed=False)
    _, counts = np.unique(labels, return_counts=True)
    counts1 = -np.sort(-counts)

71.3 ms ± 6.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# 3) Using scipy.connected_components with sparse matrix

In [5]:
%%timeit

for _ in range(100):
    _, labels = connected_components(A, directed=False)
    _, counts = np.unique(labels, return_counts=True)
    counts1 = -np.sort(-counts)

37.5 ms ± 8.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# 4) Using networkx

In [6]:
%%timeit

for _ in range(100):
    aa = nx.from_numpy_matrix(a)
    b = nx.components.connected_components(aa)
    counts = np.array([len(c) for c in b])
    counts3 = -np.sort(-counts)

54.4 ms ± 994 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# 5) Using iGraph

In [7]:
%%timeit

for _ in range(100):
    graph = g.Graph.Adjacency(a.tolist())
    counts = np.array(graph.clusters().sizes())
    counts5 = -np.sort( -counts )

32.6 ms ± 1.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# 6) Using GraphTool

In [8]:
%%timeit

for _ in range(100):
    gr = gt.Graph(directed=False)
    gr.add_edge_list( np.transpose(a.nonzero()) )
    comp, hist = gt.topology.label_components(gr)
    counts6 = -np.sort( -np.array(hist) )

33.5 ms ± 972 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
